In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import utils
import random

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
# hyperparameters
batch_iterations = 500
batch_size = 27
full_iterations = 300
learning_rate = 0.001
reg_eta = 0.001

# dimensionalities
dim_lstm = 300
dim_word = 300
dim_aspect = 5
dim_aspect_embedding = 300
dim_sentence = 80
dim_polarity = 3

# setup utils object
isSample = False
u = utils.UTILS(batch_size, dim_sentence, dim_polarity, isSample)

In [3]:
# define tf placeholders
X = tf.placeholder(tf.int32, [None, dim_sentence])
y = tf.placeholder(tf.float32, [None, dim_polarity])
seqlen = tf.placeholder(tf.int32, [None])
aspects = tf.placeholder(tf.int32, [None])

In [4]:
# define tf variables
with tf.variable_scope('aspect_embedding_vars', reuse = tf.AUTO_REUSE):
    fw_va = tf.get_variable(
        name = 'aspect_matrix_forward_Va',
        shape = [dim_aspect, dim_aspect_embedding],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    bk_va = tf.get_variable(
        name = 'aspect_matrix_backward_Va',
        shape = [dim_aspect, dim_aspect_embedding],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wv = tf.get_variable(
        name = 'aspect_Wv',
        shape = [dim_aspect_embedding * 2, dim_aspect_embedding * 2],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
with tf.variable_scope('attention_vars', reuse = tf.AUTO_REUSE):
    wh = tf.get_variable(
        name = 'M_tanh_Wh',
        shape = [dim_lstm * 2, dim_lstm * 2],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    w = tf.get_variable(
        name = 'alpha_softmax_W',
        shape = [(dim_lstm + dim_aspect_embedding) * 2, 1],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wp = tf.get_variable(
        name = 'hstar_tanh_Wp',
        shape = [dim_lstm * 2, dim_lstm * 2],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wx = tf.get_variable(
        name = 'hstar_tanh_Wx',
        shape = [dim_lstm * 2, dim_lstm * 2],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
with tf.variable_scope('output_softmax_vars', reuse = tf.AUTO_REUSE):
    ws = tf.get_variable(
        name = 'y_softmax_Ws',
        shape = [dim_lstm * 2, dim_polarity],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    bs = tf.get_variable(
        name = 'y_softmax_Bs',
        shape = [dim_polarity],
        initializer = tf.random_uniform_initializer(-0.003, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )    

Instructions for updating:
Use the retry module or similar alternatives.


In [12]:
# define lstm model
def dynamic_lstm(inputs, seqlen, aspects):
#     inputs = tf.nn.dropout(inputs, keep_prob=1.0)
    with tf.name_scope('lstm_model'):
        # slice the corresponding vai from va
        fw_vai = tf.gather(fw_va, aspects) # batch_size x dim_aspect_embedding
        bk_vai = tf.gather(bk_va, aspects) # batch_size x dim_aspect_embedding
#         # concatenate vai to inputs
#         vai_en = [vai for i in range(dim_sentence)]
#         vai_en = tf.stack(vai_en, axis = 1) # batch_size x dim_sentence x dim_aspect_embedding
#         inputs = tf.concat([inputs, vai_en], 2)
        forward_lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        backward_lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        H, states = tf.nn.bidirectional_dynamic_rnn(
            forward_lstm_cell,
            backward_lstm_cell,
            inputs = inputs,
            sequence_length = seqlen,
            dtype = tf.float32,
            scope = 'bilstm'
        )
        fw, bk = H
        bk = tf.reverse_sequence(bk, tf.cast(seqlen, tf.int64), seq_dim=1)
        H = tf.concat(H, 2)
        size = tf.shape(H)[0]
        wv_vai = tf.matmul(tf.concat([fw_vai, bk_vai], 1), wv) # batch_size x (dim_aspect_embedding * 2)
        # stacking Wv x Va along sentence length
        wv_vai = [wv_vai for i in range(dim_sentence)]
        wv_vai_en = tf.stack(wv_vai, axis = 1) # batch_size x dim_sentence x (dim_aspect_embedding * 2)
        wv_vai_en = tf.reshape(wv_vai_en, [-1, dim_aspect_embedding * 2]) # (batch_size * dim_sentence) x (dim_aspect_embedding * 2)
        H_1 = tf.reshape(H, [-1, dim_lstm * 2]) # (batch_size * dim_sentence) x (dim_lstm * 2)
        wh_H = tf.matmul(H_1, wh) # (batch_size * dim_sentence) x (dim_lstm * 2)
        # concatenate wh_H and wv_va_En for inputting to tanh
        wh_H_wv_vai_en = tf.concat([wh_H, wv_vai_en], 1) # (batch_size * dim_sentence) x [(dim_lstm + dim_aspect_embedding) * 2]
        M = tf.tanh(wh_H_wv_vai_en) # (batch_size * dim_sentence) x [(dim_lstm + dim_aspect_embedding) * 2]
        alpha = tf.nn.softmax(tf.matmul(M, w)) # (batch_size * dim_sentence)
        alpha = tf.reshape(alpha, [-1, 1, dim_sentence]) # batch_size x 1 x dim_sentence
        index = tf.range(0, size) * dim_sentence + seqlen - 1 # batch_size
        hn = tf.gather(tf.reshape(H, [-1, dim_lstm * 2]), index)  # batch_size x (dim_lstm * 2)
        r = tf.reshape(tf.matmul(alpha, H), [-1, dim_lstm * 2]) # batch_size x (dim_lstm * 2)
        h_star = tf.tanh(tf.matmul(r, wp) + tf.matmul(hn, wx)) # batch_size x (dim_lstm * 2)
        predict = tf.matmul(h_star, ws) + bs # batch x dim_polarity
    return predict


SyntaxError: invalid syntax (<ipython-input-12-82446be21db6>, line 2)

In [6]:
# define operations
# tf.reset_default_graph()
pred = dynamic_lstm(tf.nn.embedding_lookup(u.gloveDict, X), seqlen, aspects)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
# full dataset training
test_X, test_y, test_seqlen, test_aspects = u.getData('test')
train_X, train_y, train_seqlen, train_aspects = u.getData('train')
with tf.Session() as sess:
    sess.run(init)
    for i in range(full_iterations):
        sess.run(optimizer, feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, aspects: train_aspects})
#         if i > 0 and i % 4 == 0:
        loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, aspects: train_aspects})
        print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
        loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen, aspects: test_aspects})
        print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

step: 0, train loss: 0.9888515, train accuracy: 0.619386
step: 0, test loss: 0.94983697, test accuracy: 0.6752312
step: 1, train loss: 0.95584023, train accuracy: 0.619386
step: 1, test loss: 0.85203785, test accuracy: 0.6752312
step: 2, train loss: 0.939962, train accuracy: 0.619386
step: 2, test loss: 0.8291558, test accuracy: 0.6752312
step: 3, train loss: 0.9336038, train accuracy: 0.619386
step: 3, test loss: 0.8411798, test accuracy: 0.6752312
step: 4, train loss: 0.93572277, train accuracy: 0.619386
step: 4, test loss: 0.8596165, test accuracy: 0.6752312
step: 5, train loss: 0.9174634, train accuracy: 0.619386
step: 5, test loss: 0.84507376, test accuracy: 0.6752312
step: 6, train loss: 0.9161699, train accuracy: 0.619386
step: 6, test loss: 0.8404913, test accuracy: 0.6752312
step: 7, train loss: 0.92303145, train accuracy: 0.619386
step: 7, test loss: 0.8434742, test accuracy: 0.6752312
step: 8, train loss: 0.91307676, train accuracy: 0.619386
step: 8, test loss: 0.83405215, t

KeyboardInterrupt: 

In [ ]:
# batch training
test_X, test_y, test_seqlen, test_aspects = u.getData('test')
with tf.Session() as sess:
    sess.run(init)
    for i in range(batch_iterations):
        batch_X, batch_y, batch_seqlen, batch_aspects = u.nextBatch(batch_size)
        sess.run(optimizer, feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen, aspects: batch_aspects})
        if i > 0 and i % 4 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen, aspects: batch_aspects})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen, aspects: test_aspects})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

In [ ]:
tf.InteractiveSession()
u.trainEncode

In [ ]:
u.trainData.head()

In [8]:
test_X, test_y, test_seqlen, test_aspects = u.getData('test')

In [10]:
len(test_X)

1120

In [11]:
test_X

[array([1557, 2363,  456, 3057, 3201,  249, 3563, 2762,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0]),
 array([3862,  104, 4062, 2948, 3838,  104,  219, 1005, 1557, 2028, 4264,
         968, 1352, 1557, 2334, 2762,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          